In [1]:
import mysql.connector
import psycopg2
import csv

# --- Konfigurasi Koneksi Database ---
# Ganti dengan detail koneksi Anda
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'thyroid_db'  # Ganti dengan nama database MySQL Anda
}

postgres_config = {
    'host': 'localhost',
    'user': 'postgres',
    'password': 'abdi19',
    'database': 'postgres'
}

def clean_value(value):
    """Mengonversi '?' menjadi None, selain itu kembalikan nilai aslinya."""
    return None if value.strip() == '?' else value.strip()

def process_and_insert_data(filepath):
    """Membaca file data dan memasukkannya ke dalam MySQL dan PostgreSQL."""
    
    try:
        # Membuat koneksi
        mysql_conn = mysql.connector.connect(**mysql_config)
        postgres_conn = psycopg2.connect(**postgres_config)
        
        mysql_cursor = mysql_conn.cursor()
        postgres_cursor = postgres_conn.cursor()
        
        print("Koneksi database berhasil.")

        with open(filepath, 'r') as file:
            reader = csv.reader(file)
            
            for row in reader:
                # Membersihkan nilai '?' menjadi None
                cleaned_row = [clean_value(v) for v in row]

                # 1. Pisahkan data untuk setiap database
                
                # Data untuk MySQL (Label dan Numerik)
                # Kolom: class, age, TSH, T3, TT4, T4U, FTI, TBG
                mysql_data = (
                    cleaned_row[0],   # class
                    cleaned_row[1],   # age
                    cleaned_row[15],  # TSH
                    cleaned_row[17],  # T3
                    cleaned_row[19],  # TT4
                    cleaned_row[21],  # T4U
                    cleaned_row[23],  # FTI
                    cleaned_row[25]   # TBG
                )
                
                # Data untuk PostgreSQL (Kategorikal)
                # Kolom: sex, on_thyroxine, ..., TBG_measured
                postgres_data = (
                    cleaned_row[2],   # sex
                    cleaned_row[3],   # on_thyroxine
                    cleaned_row[4],   # query_on_thyroxine
                    cleaned_row[5],   # on_antithyroid_medication
                    cleaned_row[6],   # thyroid_surgery
                    cleaned_row[7],   # query_hypothyroid
                    cleaned_row[8],   # query_hyperthyroid
                    cleaned_row[9],   # pregnant
                    cleaned_row[10],  # sick
                    cleaned_row[11],  # tumor
                    cleaned_row[12],  # lithium
                    cleaned_row[13],  # goitre
                    cleaned_row[14],  # TSH_measured
                    cleaned_row[16],  # T3_measured
                    cleaned_row[18],  # TT4_measured
                    cleaned_row[20],  # T4U_measured
                    cleaned_row[22],  # FTI_measured
                    cleaned_row[24]   # TBG_measured
                )

                # 2. Masukkan data ke MySQL terlebih dahulu untuk mendapatkan ID
                sql_mysql = """
                    INSERT INTO hypothyroid_numerical (class, age, TSH, T3, TT4, T4U, FTI, TBG)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                mysql_cursor.execute(sql_mysql, mysql_data)
                
                # Dapatkan ID dari baris yang baru saja dimasukkan
                last_id = mysql_cursor.lastrowid

                # 3. Masukkan data ke PostgreSQL menggunakan ID dari MySQL
                # Menambahkan ID ke awal tuple data postgres
                postgres_data_with_id = (last_id,) + postgres_data
                
                sql_postgres = """
                    INSERT INTO hypothyroid_categorical (id, sex, on_thyroxine, query_on_thyroxine, 
                                                         on_antithyroid_medication, thyroid_surgery, query_hypothyroid, 
                                                         query_hyperthyroid, pregnant, sick, tumor, lithium, goitre, 
                                                         TSH_measured, T3_measured, TT4_measured, T4U_measured, 
                                                         FTI_measured, TBG_measured)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                postgres_cursor.execute(sql_postgres, postgres_data_with_id)

        # Commit perubahan ke kedua database
        mysql_conn.commit()
        postgres_conn.commit()
        
        print("Semua data berhasil dimasukkan.")

    except (Exception, mysql.connector.Error, psycopg2.Error) as error:
        print(f"Error: {error}")

    finally:
        # Tutup koneksi
        if 'mysql_conn' in locals() and mysql_conn.is_connected():
            mysql_cursor.close()
            mysql_conn.close()
            print("Koneksi MySQL ditutup.")
        if 'postgres_conn' in locals() and not postgres_conn.closed:
            postgres_cursor.close()
            postgres_conn.close()
            print("Koneksi PostgreSQL ditutup.")

# Pastikan file 'hypothyroid.data' berada di folder yang sama dengan skrip ini
process_and_insert_data("C:\\Dokumen\\PSD\\dataset\\hypothyroid.data")

Error: 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)
